In [1]:
import os
import image_processing
import cupy as cp

In [2]:
file_path = "data/lidar/Job1051007_34077_04_88.laz"

min_x_extent = 712160
max_x_extent = 712230
min_y_extent = 33100
max_y_extent = 33170

grid_gen = image_processing.GridGenerator(
    file_path, min_x_extent, max_x_extent, min_y_extent, max_y_extent
)

resolution = 0.05  # meters

pts_array = grid_gen.create_point_array()
grid_x, grid_y, grid_z = grid_gen.gen_grid(resolution, z=pts_array)

Directory to store grids created: generated_grids


In [3]:
grid_z_gpu = cp.asarray(grid_z)

mapper = image_processing.DepthMapper(grid_z_gpu)

subfolder_path = 'data/CB_03_20220909212130_20220910054130'
labels_rects_zarr_folder = os.path.join(subfolder_path, "zarr", "labels_rects")

# Check if the labels_rects folder exists
if os.path.exists(labels_rects_zarr_folder):
    # print(f"Processing folder {subfolder}.")
    depth_map_zarr_save_dir = os.path.join(
        subfolder_path, "zarr", "depth_maps_updated"
    )
    pond_edge_elev_plot_dir = os.path.join(subfolder_path, 'plots', 'edge_histograms')
    mapper.process_depth_maps(labels_rects_zarr_folder, depth_map_zarr_save_dir, pond_edge_elev_plot_dir)

In [ ]:
import zarr
import geopandas as gpd
import matplotlib.pyplot as plt
import contextily as ctx
from pathlib import Path
import rioxarray
import xarray as xr
import numpy as np
from affine import Affine
import cmocean

# Imports needed for the manual colorbar
from matplotlib import cm
from matplotlib.colors import Normalize


In [ ]:
def plot_flood_depth_map(depth_array, min_x, max_x, min_y, max_y,
                                 resolution_m=0.05, bbox_crs='EPSG:32119', output_folder='figures'):
    """
    Plots a flood numpy array, correcting for a "bottom-left" array origin by
    flipping the array vertically before georeferencing.

    Args:
        numpy_array (np.ndarray): The raw numpy array, assumed to have a (0,0) origin
                                  at the bottom-left.
        ... (other args are the same) ...
    """

    # 1. Build and georeferenced the DataArray.
    # --- THE CRITICAL FIX BASED ON YOUR INSIGHT ---
    # Vertically flip the array to convert from a "bottom-left" origin to the
    # "top-left" origin expected by geospatial raster standards.
    data_for_xarray = np.flipud(depth_array).astype(float)

    transform = Affine(resolution_m, 0.0, min_x, 0.0, -resolution_m, max_y)
    da_hmax = xr.DataArray(
        data=data_for_xarray,
        dims=["y", "x"],
        name='flood_depth'
    )
    da_hmax.rio.write_crs(bbox_crs, inplace=True)
    da_hmax.rio.write_transform(transform, inplace=True)
    da_hmax.rio.write_nodata(np.nan, inplace=True)

    # 2. Reproject the raster to Web Mercator.
    da_hmax_mercator = da_hmax.rio.reproject(3857)

    # 3. Create the plot axes
    fig, ax = plt.subplots(figsize=(10, 10))

    # 4. Get the spatial bounds and data from the reprojected raster.
    minx, miny, maxx, maxy = da_hmax_mercator.rio.bounds()
    data_to_plot = da_hmax_mercator.to_numpy()

    # 5. Set the axis limits.
    ax.set_xlim(minx, maxx)
    ax.set_ylim(miny, maxy)

    # 6. Add the basemap FIRST.
    ctx.add_basemap(ax, source=ctx.providers.Esri.WorldImagery, zorder=1)

    vmin = 0
    vmax = 0.25
    
    # 7. Use ax.imshow() to plot the data ON TOP.
    im = ax.imshow(
        data_to_plot,
        extent=(minx, maxx, miny, maxy),
        cmap=cmocean.cm.dense,
        vmin=vmin,
        vmax=vmax,
        alpha=0.7,
        interpolation='none',
        zorder=10
    )

    ax.text(
            0.05,
            0.95,
            f"Spatial Extent ($m^2$): {round((np.sum(~np.isnan(depth_array))) * 0.0025, 2)}",
            transform=ax.transAxes,
            fontsize=12,
            verticalalignment="top",
            bbox=dict(facecolor="white", alpha=0.8, edgecolor="black"),
        )
    
    # 8. Manually create a colorbar.
    cbar = fig.colorbar(im, ax=ax, shrink=0.6, aspect=30)
    cbar.set_label('Depth (m)')
    
    # 9. Clean up and save
    png_path = Path(output_folder) / "flood_map_final_depth_95.png"
    png_path.parent.mkdir(parents=True, exist_ok=True)

    ax.set_title('')
    ax.set_axis_off()
    plt.tight_layout()

    plt.savefig(png_path, dpi=300, bbox_inches='tight', pad_inches=0)
    plt.close(fig)
    print(f"Plot created successfully and saved to {png_path}.")

In [ ]:

def plot_flood_wse_map(depth_array, min_x, max_x, min_y, max_y,
                                 resolution_m=0.05, bbox_crs='EPSG:32119', output_folder='figures'):
    """
    Plots a flood numpy array, correcting for a "bottom-left" array origin by
    flipping the array vertically before georeferencing.

    Args:
        numpy_array (np.ndarray): The raw numpy array, assumed to have a (0,0) origin
                                  at the bottom-left.
        ... (other args are the same) ...
    """

    # 1. Build and georeferenced the DataArray.
    # --- THE CRITICAL FIX BASED ON YOUR INSIGHT ---
    # Vertically flip the array to convert from a "bottom-left" origin to the
    # "top-left" origin expected by geospatial raster standards.
    data_for_xarray = np.flipud(depth_array).astype(float)

    transform = Affine(resolution_m, 0.0, min_x, 0.0, -resolution_m, max_y)
    da_hmax = xr.DataArray(
        data=data_for_xarray,
        dims=["y", "x"],
        name='flood_depth'
    )
    da_hmax.rio.write_crs(bbox_crs, inplace=True)
    da_hmax.rio.write_transform(transform, inplace=True)
    da_hmax.rio.write_nodata(np.nan, inplace=True)

    # 2. Reproject the raster to Web Mercator.
    da_hmax_mercator = da_hmax.rio.reproject(3857)

    # 3. Create the plot axes
    fig, ax = plt.subplots(figsize=(10, 10))

    # 4. Get the spatial bounds and data from the reprojected raster.
    minx, miny, maxx, maxy = da_hmax_mercator.rio.bounds()
    data_to_plot = da_hmax_mercator.to_numpy()

    # 5. Set the axis limits.
    ax.set_xlim(minx, maxx)
    ax.set_ylim(miny, maxy)

    # 6. Add the basemap FIRST.
    ctx.add_basemap(ax, source=ctx.providers.Esri.WorldImagery, zorder=1)
    
    vmin=np.nanmin(data_to_plot)
    vmax=np.nanmax(data_to_plot)
    
    # 7. Use ax.imshow() to plot the data ON TOP.
    im = ax.imshow(
        data_to_plot,
        extent=(minx, maxx, miny, maxy),
        cmap='Blues',
        alpha=0.7,
        interpolation='none',
        zorder=10
    )

    ax.text(
            0.05,
            0.95,
            f"Spatial Extent ($m^2$): {round((np.sum(~np.isnan(depth_array))) * 0.0025, 2)}",
            transform=ax.transAxes,
            fontsize=12,
            verticalalignment="top",
            bbox=dict(facecolor="white", alpha=0.8, edgecolor="black"),
        )
    
    # 8. Manually create a colorbar.
    norm = Normalize(vmin=np.nanmin(data_to_plot), vmax=np.nanmax(data_to_plot))
    # Create the colorbar
    cbar = fig.colorbar(cm.ScalarMappable(norm=norm, cmap='Blues'), ax=ax, shrink=0.6, aspect=30)
    cbar.set_label('Water Surface Elevation (m)')
    
    # 9. Clean up and save
    png_path = Path(output_folder) / "flood_map_final_wse_95.png"
    png_path.parent.mkdir(parents=True, exist_ok=True)

    ax.set_title('')
    ax.set_axis_off()
    plt.tight_layout()

    plt.savefig(png_path, dpi=300, bbox_inches='tight', pad_inches=0)
    plt.close(fig)
    print(f"Plot created successfully and saved to {png_path}.")

In [ ]:
min_x_extent = 712160
max_x_extent = 712230
min_y_extent = 33100
max_y_extent = 33170

zarr_store_path = 'data/CB_03_20220909212130_20220910054130/zarr/depth_maps_updated/CAM_CB_03_20220910051234_predseg_labels_rectified_wse_map_95_perc'

img_store = zarr.open(zarr_store_path, mode="r")
depth_map = img_store[:]

plot_flood_wse_map(depth_map, min_x_extent, max_x_extent, min_y_extent, max_y_extent, bbox_crs='EPSG:32119', output_folder='figures')